In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [3]:
twitter_df.index

DatetimeIndex(['2017-01-20 12:31:53', '2017-01-20 17:51:25',
               '2017-01-20 17:51:58', '2017-01-20 17:52:45',
               '2017-01-20 17:53:17', '2017-01-20 17:54:00',
               '2017-01-20 17:54:36', '2017-01-20 17:55:44',
               '2017-01-20 17:58:24', '2017-01-20 18:00:43',
               ...
               '2019-06-19 14:18:11', '2019-06-19 14:18:12',
               '2019-06-19 14:20:04', '2019-06-19 15:09:18',
               '2019-06-19 19:48:02', '2019-06-19 19:48:04',
               '2019-06-19 20:29:24', '2019-06-19 20:39:28',
               '2019-06-19 23:01:59', '2019-06-20 00:12:31'],
              dtype='datetime64[ns]', name='date', length=8442, freq=None)

In [4]:
#################### TIMEINDEX FUNCTIONS #####################
def get_day_window_size_from_freq(dataset):#, freq='CBH'):
    
    if dataset.index.freq == custom_BH_freq():
        return 7
    
    if dataset.index.freq=='T':
        day_window_size = 1440
    elif dataset.index.freq=='BH':
        day_window_size = 8
    elif dataset.index.freq=='CBH':
        day_window_size = 7
    elif dataset.index.freq=='B':
        day_window_size=1
    elif dataset.index.freq=='D':
        day_window_size=1
        
    else:
        raise Exception('dataset freq=None')
        
    return day_window_size
    

def custom_BH_freq():
    import pandas as pd
    CBH = pd.tseries.offsets.CustomBusinessHour(start='09:30',end='16:30')
    return CBH
    
    
def  set_timeindex_freq(ive_df, col_to_fill=None, freq='CBH',fill_method='ffill',
                        verbose=3): #set_tz=True,
    
    import pandas as pd
    import numpy as np
    
    
    if verbose>1:
        # print(f"{'Index When:':>{10}}\t{'Freq:':>{20}}\t{'Index Start:':>{40}}\t{'Index End:':>{40}}")
        print(f"{'Index When:'}\t{'Freq:'}\t{'Index Start'}\t\t{'Index End:'}")
        print(f"Pre-Change\t{ive_df.index.freq}\t{ive_df.index[0]}\t{ive_df.index[-1]}")
        
    
    if freq=='CBH':
        freq=custom_BH_freq()
#         start_idx = 
        
    # Change frequency to freq
    ive_df = ive_df.asfreq(freq,)#'min')
    
    #     # Set timezone
    #     if set_tz==True:
    #         ive_df.tz_localize()
    #         ive_df.index = ive_df.index.tz_convert('America/New_York')
    
    # Report Success / Details
    if verbose>1:
        print(f"Post-Change\t{ive_df.index.freq}\t{ive_df.index[0]}\t{ive_df.index[-1]}")


    ## FILL AND TRACK TIMEPOINTS WITH MISSING DATA    
    
    # Helper Function for adding column to track the datapoints that were filled
    def check_null_times(x):
        import numpy as np
        if np.isnan(x):
            return True
        else:
            return False

    ## CREATE A COLUMN TO TRACK ROWS TO BE FILLED
    # If col_to_fill provided, use that column to create/judge ive_df['filled_timebin'] 
    if col_to_fill!=None:
        ive_df['filled_timebin'] = ive_df[col_to_fill].apply(lambda x: check_null_times(x))#True if ive_df.isna().any()
        
    # if not provided, use all columns and sum results
    elif col_to_fill == None:
        # Prefill fol with 0's
        ive_df['filled_timebin']=0
        
        # loop through all columns and add results of check_null_times from each loop
    for col in ive_df.columns:
        if ive_df[col].dtypes=='float64':
            #ive_df['filled_timebin'] = ive_df[target_col].apply(lambda x: check_null_times(x))#True if ive_df.isna().any()
            curr_filled_timebin_col = ive_df[col].apply(lambda x: check_null_times(x))#True if ive_df.isna().any() 

            # add results
            ive_df['filled_timebin'] +=  curr_filled_timebin_col
            
    ive_df['filled_timebin'] = ive_df['filled_timebin'] >0
            
    ## FILL IN NULL VALUES
    ive_df.fillna(method=fill_method, inplace=True)

    # Report # filled
    if verbose>0:
        check_fill = ive_df.loc[ive_df['filled_timebin']>0]
        print(f'\nFilled {len(check_fill==True)}# of rows using method {fill_method}')
    
    # Report any remaning null values
    if verbose>0:
        res = ive_df.isna().sum()
        if res.any():
            print(f'Cols with Nulls:')
            print(res[res>0])
        else:
            print('No Remaining Null Values')   
            
    # display header
    if verbose>2:
        display(ive_df.head())
    
    return ive_df


# Helper Function for adding column to track the datapoints that were filled
def check_null_times(x):
    import numpy as np
    if np.isnan(x):
        return True
    else:
        return False

In [57]:
del(twitter_df)

In [58]:
try: twitter_df
except NameError: twitter_df = None
    
if twitter_df is None:
    print('loading twitter_df')
    twitter_df = pd.read_csv('data/trump_twitter_archive_df.csv', encoding='utf-8')
    twitter_df.drop('Unnamed: 0',axis=1,inplace=True)
    
    twitter_df['date']  = pd.to_datetime(twitter_df['date'])
    twitter_df.set_index('date',inplace=True,drop=False)

    # Fill in missing values before merging with stock data
    twitter_df.fillna('', inplace=True)
    twitter_df.sort_index(ascending=True, inplace=True)

    # Check header and daterange of index
    display(twitter_df.head(2))
    twitter_df.index[[0,-1]]
else:
    print('twitter_df already exists.')

loading twitter_df


,source,content,date,retweet_count,favorite_count,is_retweet,id_str,case_ratio,content_raw,has_RT,...,content_stopped,tokens_stopped,hashtag_strings,mention_strings,sentiment_scores,compound_score,sentiment_class,neg,neu,pos
date,,,,,,,,,,,,,,,,,,,,,
2017-01-20 12:31:53,Twitter for Android,It all begins today! I will see you at 11:00 A...,2017-01-20 12:31:53,70523,268372,False,822421390125043713,0.33036,It all begins today! I will see you at 11:00 A...,False,...,begins today see swearing movement continues w...,"['begins', 'today', 'see', 'swearing', 'moveme...",,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos,0.0,1.000,0.000
2017-01-20 17:51:25,Twitter for iPhone,Today we are not merely transferring power fro...,2017-01-20 17:51:25,20125,109640,False,822501803615014918,0.01481,Today we are not merely transferring power fro...,False,...,today merely transferring power one administra...,"['today', 'merely', 'transferring', 'power', '...",,,"{'neg': 0.0, 'neu': 0.919, 'pos': 0.081, 'comp...",0.2144,pos,0.0,0.919,0.081


# How to merge stock-market-business hours tweets?
- use `custom_BH_freq` to create a new column that attempts to re-cast the index as this frequency.
    - separate the index off first to another column/dataframe.
    - create a second column that rounds/floors/ceils the timestamp values to the correct business hour
    - **NOTE**: -filling? revisit

In [59]:
twitter_df.index

DatetimeIndex(['2017-01-20 12:31:53', '2017-01-20 17:51:25',
               '2017-01-20 17:51:58', '2017-01-20 17:52:45',
               '2017-01-20 17:53:17', '2017-01-20 17:54:00',
               '2017-01-20 17:54:36', '2017-01-20 17:55:44',
               '2017-01-20 17:58:24', '2017-01-20 18:00:43',
               ...
               '2019-06-19 14:18:11', '2019-06-19 14:18:12',
               '2019-06-19 14:20:04', '2019-06-19 15:09:18',
               '2019-06-19 19:48:02', '2019-06-19 19:48:04',
               '2019-06-19 20:29:24', '2019-06-19 20:39:28',
               '2019-06-19 23:01:59', '2019-06-20 00:12:31'],
              dtype='datetime64[ns]', name='date', length=8442, freq=None)

In [55]:
len(twitter_df), len(twitter_df.index)

(8442, 8442)

In [61]:
CBH = custom_BH_freq()
df = twitter_df[['date','id_str']]
df.columns=['date_to_bin','id_str']
# df['date'] = twitter_df.index
df.head()

,date_to_bin,id_str
date,,
2017-01-20 12:31:53,2017-01-20 12:31:53,822421390125043713
2017-01-20 17:51:25,2017-01-20 17:51:25,822501803615014918
2017-01-20 17:51:58,2017-01-20 17:51:58,822501939267141634
2017-01-20 17:52:45,2017-01-20 17:52:45,822502135233384448
2017-01-20 17:53:17,2017-01-20 17:53:17,822502270503972872
